In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import statistics 

In [2]:
x_train_data = pd.read_csv('/home/asma/Documents/Intelligent Systems/Question 4/train_data.csv', header=None)
x_test_data = pd.read_csv('/home/asma/Documents/Intelligent Systems/Question 4/test_data.csv', header=None)
y_train_data = pd.read_csv('/home/asma/Documents/Intelligent Systems/Question 4/train_labels.csv', header=None)
y_test_data = pd.read_csv('/home/asma/Documents/Intelligent Systems/Question 4/test_labels.csv', header=None)

In [3]:
y_train_data.rename(columns = {0:'label'}, inplace = True)
y_test_data.rename(columns = {0:'label'}, inplace = True)

In [1]:
w = 0
for i in range(10):
    for j in range(10):
        w =w+1
w        

100

In [4]:
print(x_train_data.shape)
print(y_train_data.shape)
print(x_test_data.shape)
print(y_test_data.shape)

(10000, 182)
(10000, 1)
(3500, 182)
(3500, 1)


In [5]:
y_train_data.nunique()

label    10
dtype: int64

In [6]:
y_test_data['label'].value_counts()

3.0    375
8.0    369
2.0    368
4.0    365
5.0    350
1.0    346
9.0    345
7.0    331
0.0    329
6.0    322
Name: label, dtype: int64

In [120]:
def KNN(k, test_row, x_train, y_train, meter):
    df =  pd.DataFrame()
    
    if (meter == 'euclidean'):
        df['distance'] = ((x_train -test_row)**2).sum(axis=1).apply(lambda x: math.sqrt(x))
        
    if (meter == 'd1'):
        df['distance'] = (x_train - test_row).max(axis=1)
    
    if (meter == 'd2'):
        df['distance'] = (abs(x_train - test_row)).sum(axis=1)
    
    df['label'] = y_train
    df = df.sort_values(by= 'distance')   
    neighbours = df[:k]
    predicted_class = neighbours.groupby('label').size().idxmax()
    
    return predicted_class

In [72]:
def main(k , x_test, x_train, y_test, y_train, meter): 
    
    y_predict = x_test.apply(lambda row : KNN(k, row, x_train, y_train, meter), axis=1)
    predict_df = pd.DataFrame(y_predict, columns={'predicted_class'})
    correct = (predict_df['predicted_class'] == y_test_data['label'])
    
    return correct.sum() / len(x_test) * 100

In [73]:
main(2, x_test_data, x_train_data, y_test_data, y_train_data, 'euclidean')

14.314285714285715

In [77]:
main(5, x_test_data, x_train_data, y_test_data, y_train_data, 'euclidean')

14.399999999999999

In [13]:
main(10, x_test_data, x_train_data, y_test_data, y_train_data, 'euclidean')

13.82857142857143

In [14]:
main(50, x_test_data, x_train_data, y_test_data, y_train_data, 'euclidean')

14.285714285714285

مشاهد می کنیم که بالاترین دقت در حالتی که تعداد همسایه ها ۵ تا است اتفاق میافتد. زمانی که تعدا همسایه ها ۲ است الگوریتم کلاس را تنها بر اساس دو تا از نزدیک ترین مشاهده ها پیش بینی میکند که ممکن است نویز زیاد داشته باشد و همچنین در خیلی از مواقع ممکن است هر کدام از این دو همسایه کلاس متفاوتی داشته باشند و کلاسی که بیشتر در بین همسایه ها تکرار شده عملن بی معنی خواهد بود. 
این مشکل برای تمام عددهای زوج دیگر یعنی ۱۰ و ۵۰ نیز پیش خواهد امد. اگر تعداد همسایه ها خیلی زیادنیزباشد دقت مذل کم خواهد بود چون در این حالت نمونه هایی که خیلی دور هم هستند ممکن است در نظر گرفته شوند.
همچنین مشاهده میشود که دقت مدل بسیار پایین است. حدس زده میشود این مشکل به دلیل بازه های بسیار متفاوت مقدارهای داده شده است و با نرمال کردن داده ها یعنی بردن تمام داده ها به بازه ی ۰ .و ۱ این مشکل حل شود.


In [52]:
def normalize(df):
    return (df - df.min())/(df.max() - df.min())

In [74]:
main(2, normalize(x_test_data), normalize(x_train_data), y_test_data, y_train_data, 'euclidean')

73.68571428571428

In [75]:
main(5, normalize(x_test_data), normalize(x_train_data), y_test_data, y_train_data, 'euclidean')

76.14285714285714

In [76]:
main(10, normalize(x_test_data), normalize(x_train_data), y_test_data, y_train_data, 'euclidean')

74.45714285714286

In [78]:
main(50, normalize(x_test_data), normalize(x_train_data), y_test_data, y_train_data, 'euclidean')

68.28571428571428

مشاهده میشود که با نرمال کردن مقادیر داده ها دقت مدل به طرز قابل توجهی افزایش میابد و همچنان بهترین دقت در حالتی که تعداد همسایه ها ۵ تا باشد رخ میدهد.

In [121]:
main(5, normalize(x_test_data), normalize(x_train_data), y_test_data, y_train_data, 'd1')

49.25714285714285

In [122]:
main(5, normalize(x_test_data), normalize(x_train_data), y_test_data, y_train_data, 'd2')

78.97142857142858

با استفاده از معیار د۱ دقت مدل بسیار کاهش میباد اما استفاده از نرم ۱ یا همان معیار ۲ دقت از نرم ۲ یا همان فاصله ی اقلیدسی اندکی بیشتر میشود